In [3]:
import os
import math

import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.cluster import KMeans
from math import radians, cos, sin, asin, sqrt
from sklearn.linear_model import LinearRegression
pd.options.mode.chained_assignment = None  # default='warn'

def read_data(path):
    '''Read csv files, especially when the path has Chinese
    
    Parameters
    ----------
    path: string. the path of csv file
    
    Returns
    ----------
    data
    '''
    data = pd.read_csv(path,engine='python',encoding="utf8")
    return data


def add_deviation(data):
    '''add deviation from trend yield in GDT'''
    trend_n = 30
    trend_yield = []
    for test_year in range(2004,2022):
        linear_m = LinearRegression()
        data_train_val1 = data[data['Year'].isin([i for i in range(test_year-trend_n,test_year)])]
        print(set(data_train_val1.Year))
        X99 = np.array(data_train_val1['Year']).reshape(-1,1)
        Y99 = np.array(data_train_val1['yield(t/ha)']).reshape(-1,1)

        linear_m.fit(X99, Y99)

        print('The foluma of the yield trend: yield =  {:.3f}×year + {:.3f} '
              .format(linear_m.coef_[0,0],linear_m.intercept_[0]))
        trend_yield = linear_m.coef_[0,0]*test_year + linear_m.intercept_[0]
        print(trend_yield)

        # 创建一个布尔掩码，用于筛选出 Year 列中等于 2014 的行
        mask = data['Year'] == test_year

        # 使用 loc 方法选择满足条件的行，并计算新列的值
        data.loc[mask, 'GDT_trend_yield'] = trend_yield
    return data


In [8]:
# 大豆
# 大豆数据瘦身得到输入数据
inputdata = read_data('D:/论文-产量趋势利用/数据/alldata_soybean.csv')
pic = read_data('D:/论文-产量趋势利用/数据/面积数据/soybean_PIC_resampled.csv')
soildata = pd.read_csv('D:/毕业论文/土壤数据/soil_feature.csv',engine = 'python')

inputdata = inputdata.merge(pic,on=['sta_con','Year'],how='left',validate='many_to_one')
# # 去除列
# drop_fea_list = ['Value','County','irrigated soybean area','soybean area']
# for i in ['LE','30d','shortwave','90d','RVI','Lai','Fpar','WDRVI']:
#     drop_fea_list += list(inputdata.filter(regex=i,axis = 1))
# inputdata = inputdata[inputdata.columns.drop(drop_fea_list)]
print(inputdata.index)
# 用一个州的平均灌溉水平插值PIC
inputdata.PIC = inputdata.PIC.fillna(inputdata.groupby('State').transform('mean').PIC)

# 去除行
state_list = ['illinois','indiana','iowa','michigan','minnesota','missouri','nebraska','north dakota','ohio','south dakota','wisconsin']
pattern = '|'.join(state_list)
inputdata = inputdata[inputdata['sta_con'].str.contains(pattern)]

# 去除大部分缺失行
inputdata = inputdata.loc[inputdata['7_NDVI'].dropna().index,:]
# 再填充
inputdata = inputdata.fillna(inputdata.groupby('State').transform('mean'))

# 添加土壤数据
# fill null data
soildata['i_class'] = soildata['i_class'].fillna(8)
soildata['caco3_kg_sq_m'] = soildata['caco3_kg_sq_m'].fillna(method='ffill')
# join with yield data
inputdata = inputdata.merge(soildata,on=['sta_con'],how='left',validate='many_to_one')

# 添加经纬度列并计算试验列
lon_lat = read_data('D:/论文-产量趋势利用/数据/区划数据/processed/lon_latitude.csv')
# read yield data from 1980 to now
data1980 = pd.read_csv(r'D:/论文-产量趋势利用/数据/产量数据/processed/soybean/1980年以来的大豆单产.csv',engine = 'python')
data1980 = data1980[data1980['Year'].isin([i for i in range(1980,2004,1)])]
# Make the names of states and counties lowercase.
data1980.loc[:,"State"] = data1980.loc[:,"State"].str.lower()
data1980.loc[:,"County"] = data1980.loc[:,"County"].str.lower()

# Concatenate state name and county name as sta_con.
data1980.loc[:,"sta_con"] = data1980.loc[:,"State"] + "_" + data1980.loc[:,"County"]
data1 = pd.concat([inputdata,data1980])
data1 = data1.drop(columns = ['County'])
print(data1.columns.tolist())
data1 = data1.merge(lon_lat,on=['sta_con'],how='inner',validate='many_to_one')

# 添加GDY的每年一个的趋势产量，命名为GDT_trend_yield
add_deviation(data1)
data1.to_csv('D:/毕业大论文/数据/input_soybean.csv', index=False)
data1

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            27391, 27392, 27393, 27394, 27395, 27396, 27397, 27398, 27399,
            27400],
           dtype='int64', length=27401)
['Year', 'State', 'Value', 'sta_con', 'yield(t/ha)', '7_NDVI', '7_EVI', '7_LSWI', '7_GCVI', '7_RVI', '7_SAVI', '7_WDRVI', '7_Fpar', '7_Lai', '7_ET', '7_LE', '7_LST_Day_1km', '7_LST_Night_1km', '7_spi14d', '7_spi30d', '7_spi90d', '7_eddi14d', '7_eddi30d', '7_eddi90d', '7_spei14d', '7_spei30d', '7_spei90d', '7_pdsi', '7_z', '7_sur_refl_b01', '7_sur_refl_b02', '7_sur_refl_b03', '7_sur_refl_b04', '7_sur_refl_b05', '7_sur_refl_b06', '7_sur_refl_b07', '7_total_precipitation', '7_temperature', '7_specific_humidity', '7_pressure', '7_shortwave_radiation', '7_longwave_radiation', '8_NDVI', '8_EVI', '8_LSWI', '8_GCVI', '8_RVI', '8_SAVI', '8_WDRVI', '8_Fpar', '8_Lai', '8_ET', '8_LE', '8_LST_Day_1km', '8_LST_Night_1km', '8_spi14d', '8_spi30d', '8_spi90d', '

,Year,State,Value,sta_con,yield(t/ha),7_NDVI,7_EVI,7_LSWI,7_GCVI,7_RVI,...,paws_025,ph_025,sar,texture_025,sand_025,silt_025,clay_025,Longitude,Latitude,GDT_trend_yield
0,2004,illinois,54.0,illinois_de witt,3.6288,0.255448,0.149114,-0.130004,1.190226,1.690067,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,2.638680
1,2005,illinois,56.0,illinois_de witt,3.7632,0.252361,0.132357,-0.171525,1.192192,1.675660,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,2.686000
2,2006,illinois,54.0,illinois_de witt,3.6288,0.270737,0.126103,-0.094506,1.245109,1.744906,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,2.747068
3,2007,illinois,54.0,illinois_de witt,3.6288,0.248915,0.122462,-0.082410,1.126277,1.663504,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,2.804261
4,2008,illinois,49.0,illinois_de witt,3.2928,0.260346,0.119009,0.025506,1.226170,1.704388,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,2.846738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33304,1990,wisconsin,NaN,wisconsin_iron,2.1504,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-90.242551,46.262608,NaN
33305,1980,wisconsin,NaN,wisconsin_iron,1.5456,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-90.242551,46.262608,NaN
33306,1991,wisconsin,NaN,wisconsin_price,2.4192,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-90.361272,45.680296,NaN
33307,1991,wisconsin,NaN,wisconsin_vilas,1.4784,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-89.514558,46.053366,NaN


In [10]:
# 玉米
# 玉米数据瘦身得到输入数据
inputdata = read_data('D:/论文-产量趋势利用/数据/alldata_maize.csv')
pic = read_data('D:/论文-产量趋势利用/数据/面积数据/maize_PIC_resampled.csv')
soildata = pd.read_csv('D:/毕业论文/土壤数据/soil_feature.csv',engine = 'python')

inputdata = inputdata.merge(pic,on=['sta_con','Year'],how='left',validate='many_to_one')
# # 去除列
# drop_fea_list = ['Value','County','irrigated soybean area','soybean area']
# for i in ['LE','30d','shortwave','90d','RVI','Lai','Fpar','WDRVI']:
#     drop_fea_list += list(inputdata.filter(regex=i,axis = 1))
# inputdata = inputdata[inputdata.columns.drop(drop_fea_list)]
print(inputdata.index)
# 用一个州的平均灌溉水平插值PIC
inputdata.PIC = inputdata.PIC.fillna(inputdata.groupby('State').transform('mean').PIC)

# 去除行
state_list = ['illinois','indiana','iowa','michigan','minnesota','missouri','nebraska','north dakota','ohio','south dakota','wisconsin']
pattern = '|'.join(state_list)
inputdata = inputdata[inputdata['sta_con'].str.contains(pattern)]

# 去除大部分缺失行
inputdata = inputdata.loc[inputdata['7_NDVI'].dropna().index,:]
# 再填充
inputdata = inputdata.fillna(inputdata.groupby('State').transform('mean'))

# 添加土壤数据
# fill null data
soildata['i_class'] = soildata['i_class'].fillna(8)
soildata['caco3_kg_sq_m'] = soildata['caco3_kg_sq_m'].fillna(method='ffill')
# join with yield data
inputdata = inputdata.merge(soildata,on=['sta_con'],how='left',validate='many_to_one')

# 添加经纬度列并计算试验列
lon_lat = read_data('D:/论文-产量趋势利用/数据/区划数据/processed/lon_latitude.csv')
# read yield data from 1980 to now
data1980 = pd.read_csv(r'D:/论文-产量趋势利用/数据/产量数据/processed/maize/1980年以来的玉米单产.csv',engine = 'python')
data1980 = data1980[data1980['Year'].isin([i for i in range(1980,2004,1)])]
# Make the names of states and counties lowercase.
data1980.loc[:,"State"] = data1980.loc[:,"State"].str.lower()
data1980.loc[:,"County"] = data1980.loc[:,"County"].str.lower()

# Concatenate state name and county name as sta_con.
data1980.loc[:,"sta_con"] = data1980.loc[:,"State"] + "_" + data1980.loc[:,"County"]
data1 = pd.concat([inputdata,data1980])
data1 = data1.drop(columns = ['County'])
print(data1.columns.tolist())
data1 = data1.merge(lon_lat,on=['sta_con'],how='inner',validate='many_to_one')

# 添加GDY的每年一个的趋势产量，命名为GDT_trend_yield
add_deviation(data1)
data1.to_csv('D:/毕业大论文/数据/input_maize.csv', index=False)
data1

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            31755, 31756, 31757, 31758, 31759, 31760, 31761, 31762, 31763,
            31764],
           dtype='int64', length=31765)
['Year', 'State', 'Value', 'sta_con', 'yield(t/ha)', '7_NDVI', '7_EVI', '7_LSWI', '7_GCVI', '7_RVI', '7_SAVI', '7_WDRVI', '7_Fpar', '7_Lai', '7_ET', '7_LE', '7_LST_Day_1km', '7_LST_Night_1km', '7_spi14d', '7_spi30d', '7_spi90d', '7_eddi14d', '7_eddi30d', '7_eddi90d', '7_spei14d', '7_spei30d', '7_spei90d', '7_pdsi', '7_z', '7_sur_refl_b01', '7_sur_refl_b02', '7_sur_refl_b03', '7_sur_refl_b04', '7_sur_refl_b05', '7_sur_refl_b06', '7_sur_refl_b07', '7_total_precipitation', '7_temperature', '7_specific_humidity', '7_pressure', '7_shortwave_radiation', '7_longwave_radiation', '8_NDVI', '8_EVI', '8_LSWI', '8_GCVI', '8_RVI', '8_SAVI', '8_WDRVI', '8_Fpar', '8_Lai', '8_ET', '8_LE', '8_LST_Day_1km', '8_LST_Night_1km', '8_spi14d', '8_spi30d', '8_spi90d', '

,Year,State,Value,sta_con,yield(t/ha),7_NDVI,7_EVI,7_LSWI,7_GCVI,7_RVI,...,paws_025,ph_025,sar,texture_025,sand_025,silt_025,clay_025,Longitude,Latitude,GDT_trend_yield
0,2004,illinois,181.0,illinois_de witt,11.352141,0.249488,0.144578,-0.134570,1.158571,1.666112,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,8.003437
1,2005,illinois,162.0,illinois_de witt,10.160480,0.253094,0.132772,-0.163668,1.200546,1.678357,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,8.291101
2,2006,illinois,178.0,illinois_de witt,11.163984,0.272080,0.118300,-0.090970,1.251481,1.748803,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,8.459980
3,2007,illinois,187.0,illinois_de witt,11.728455,0.252520,0.126152,-0.068340,1.150465,1.676443,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,8.637369
4,2008,illinois,179.0,illinois_de witt,11.226703,0.252110,0.115063,0.011308,1.184614,1.675581,...,5.048921,6.149656,0.03641,9.0,5.558076,67.468139,26.973785,-88.905468,40.174752,8.790439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36188,1984,wisconsin,NaN,wisconsin_forest,5.644711,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-88.770251,45.667633,NaN
36189,1983,wisconsin,NaN,wisconsin_forest,5.958306,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-88.770251,45.667633,NaN
36190,1982,wisconsin,NaN,wisconsin_forest,4.660023,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-88.770251,45.667633,NaN
36191,1981,wisconsin,NaN,wisconsin_forest,5.437738,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-88.770251,45.667633,NaN
